#### Calculate accumulated thermal units = sum (daily average temp), daily thermal time for thersholds 5-9 degree C = average temp - threshold temp, degree days for thersholds 5-9 degree C = sum (daily thermal time), photo thermal units = sum (daily thermal time * daily light)

In [196]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import matplotlib.dates as mdates
from urllib.request import urlopen
from bs4 import BeautifulSoup
import io
import requests
from sys import platform

In [197]:
if platform == "linux" or platform == "linux2":
    data_string = "/home/maria/Documents/data/pied_piper/"
    output_string = "/home/maria/Documents/output/pied_piper/"

elif platform == "win32"or platform == "win64":
    data_string = "C:/Users/maria/OneDrive/Documents/data/pied_piper/"
    output_string = "C:/Users/maria/OneDrive/Documents/output/pied_piper/"

In [191]:
def atu(df):
    daily_temp = df.groupby(by=["date"]).agg({'temp':'mean'})
    daily_temp.index = pd.to_datetime(daily_temp.index)
    daily_temp=daily_temp.sort_values(by=['date'])
    atu_df = daily_temp.loc[(daily_temp.index >= '2004-12-21')]
    atu_df['atu'] = atu_df['temp'].cumsum()
    return(atu_df)


In [190]:
def get_temp(year):
#     df_prev = pd.DataFrame({'date' : [], "time" : [], "temp" : [], "quality" : []})
    url = "https://apps.ecology.wa.gov/ContinuousFlowAndWQ/StationData/Prod/18A050/18A050_" + year + "_WTM_FM.TXT"
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()
    if year == "2005":
        text = text[131:]
    if year == "2006":
        text = text[521:-50]
    if year == "2007":
        text = text[521:-52]
    if year == "2008":
        text = text[522:-72]
    if year == "2009":
        text = text[522:-50]
    if year == "2010":
        text = text[522:-50]
    if year == "2011":
        text = text[522:-72]
    if year == "2012":
        text = text[522:-72]
    if year == "2013":
        text = text[522:-106]
    if year == "2014":
        text = text[522:-72]
        
    df = pd.read_csv(io.StringIO(text), sep='\s+', header = None, names = ["date", "time", "temp", "quality"])
#     df_new = pd.concat([df_prev,df])
#     df_prev = df
#     with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#         display(df)
    return(df)

In [189]:
def atu_combined():
    years = years = range(2005,2015)
    df_old = pd.DataFrame({'date' : [], "temp" : [], "atu" : []})
    df_old = df_old.set_index('date')
#     print(df_old)
    for year in years:
        df = get_temp(str(year))
#         print(df)
        df_atu = atu(df)
        df_new = pd.concat([df_atu,df_old])
        df_old = df_new
    return(df_old)

In [188]:
def photo_period(df_combine):
    hours_list = [] 
    for i in df_combine.index:
        date = i.strftime("%Y-%m-%d")
        url = "https://api.sunrise-sunset.org/json?lat=48.11591&lng=-123.14918&date=" + date
        response = requests.get(url)
        result = response.json()
        t = result['results']['day_length']

        (h, m, s) = t.split(':')
        hours = int(h) + int(m) * (1/60) #ignoring seconds
        hours_list.append(hours)
    df_combine['photoperiod'] = hours_list
    # df_combine.index[0].strftime("%Y-%m-%d")
    return(df_combine)

In [192]:
df_combine = atu_combined()
df_all = photo_period(df_combine)

<ipython-input-191-368b61ab38a6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atu_df['atu'] = atu_df['temp'].cumsum()


In [193]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_combine)

,temp,atu,photoperiod
date,,,
2013-10-01,7.835417,7.835417,11.683333
2013-10-02,7.979167,15.814583,11.633333
2013-10-03,8.256250,24.070833,11.583333
2013-10-04,7.928125,31.998958,11.516667
2013-10-05,8.716667,40.715625,11.466667
2013-10-06,8.770833,49.486458,11.400000
2013-10-07,8.855208,58.341667,11.350000
2013-10-08,8.652083,66.993750,11.283333
2013-10-09,7.694792,74.688542,11.233333


In [194]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_all)

,temp,atu,photoperiod
date,,,
2013-10-01,7.835417,7.835417,11.683333
2013-10-02,7.979167,15.814583,11.633333
2013-10-03,8.256250,24.070833,11.583333
2013-10-04,7.928125,31.998958,11.516667
2013-10-05,8.716667,40.715625,11.466667
2013-10-06,8.770833,49.486458,11.400000
2013-10-07,8.855208,58.341667,11.350000
2013-10-08,8.652083,66.993750,11.283333
2013-10-09,7.694792,74.688542,11.233333


In [198]:
df_all.to_csv(data_string + "temp_2005-2014_all.csv")  